In [55]:
from ortools.sat.python import cp_model
import csv


model = cp_model.CpModel()

# Constants
num_days = 7  # Number of days in a week
time_slots_per_day = 24
num_time_slots = num_days * time_slots_per_day

reader = csv.DictReader(open('/dataset/data.csv'))
print(reader)

FileNotFoundError: [Errno 2] No such file or directory: '/dataset/data.csv'

In [56]:
activities = [
    {'name': 'Workout', 'priority': 5, 'preferred_start': 16, 'duration' : 2},  # Morning workout
    {'name': 'Meeting', 'priority': 10, 'preferred_start': 12, 'duration' : 1},  # Important meeting
    {'name': 'Lunch', 'priority': 3, 'preferred_start': 12, 'duration' : 1},  # Midday lunch
    {'name': 'Study', 'priority': 7, 'preferred_start': 14, 'duration' : 2},  # Afternoon study
    {'name': 'Relax', 'priority': 1, 'preferred_start': 20, 'duration' : 3},  # Evening relaxation
]

In [49]:
 # Variables
activity_vars = {}
abs_diff_vars = {}
duration_vars = {}
for i, activity in enumerate(activities):
    # Variable for the start time of each activity
    activity_vars[activity['name']] = model.NewIntVar(0, num_time_slots - 1, f"activity_{i}")
    # Variable for the absolute difference between preferred and assigned time
    abs_diff_vars[activity['name']] = model.NewIntVar(0, num_time_slots - 1, f"abs_diff_{i}")
    # Variable for the duration of each activity
    duration_vars[activity['name']] = activity['duration']  # Duration is given in the activity

# Constraints
all_tasks = list(activity_vars.values())
model.AddAllDifferent(all_tasks)  # No overlapping activities

# Preferred time constraints and objective function
weighted_preferences = []
for activity in activities:
    activity_var = activity_vars[activity['name']]
    preferred_time = activity['preferred_start']
    priority = activity['priority']
    duration = duration_vars[activity['name']]

    # Define absolute difference using AddAbsEquality
    abs_diff_var = abs_diff_vars[activity['name']]
    model.AddAbsEquality(abs_diff_var, activity_var - preferred_time)

    # Adjust priority: Lower value closer to 0 is higher priority
    weight = (num_time_slots - priority) * abs_diff_var  # Inverse priority weighting
    weighted_preferences.append(weight)

    # Ensure the activity finishes before the end of the available time slots
    model.Add(activity_var + duration <= num_time_slots)

# Objective: Minimize weighted preferences to satisfy higher priority (lower priority number)
model.Minimize(sum(weighted_preferences))

# Solve the model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    # Create a list of scheduled activities with their start times
    scheduled_activities = []
    for activity in activities:
        start_time = solver.Value(activity_vars[activity['name']])
        duration = activity['duration']
        day = start_time // time_slots_per_day
        time = start_time % time_slots_per_day
        scheduled_activities.append({
            'name': activity['name'],
            'priority': activity['priority'],
            'scheduled_start': start_time,
            'day': day + 1,
            'time': time,
            'duration': duration
        })
    
    # Sort activities by their scheduled start time
    scheduled_activities.sort(key=lambda x: x['scheduled_start'])

    # Print sorted schedule
    print("Sorted Schedule:")
    for activity in scheduled_activities:
        print(f"Activity: {activity['name']}, Priority: {activity['priority']}, "
              f"Scheduled Start: Day {activity['day']}, Time {activity['time']}, "
              f"Duration: {activity['duration']} hours")
else:
    print("No feasible schedule found.")

Sorted Schedule:
Activity: Meeting, Priority: 10, Scheduled Start: Day 1, Time 11, Duration: 1 hours
Activity: Lunch, Priority: 3, Scheduled Start: Day 1, Time 12, Duration: 1 hours
Activity: Study, Priority: 7, Scheduled Start: Day 1, Time 14, Duration: 2 hours
Activity: Workout, Priority: 5, Scheduled Start: Day 1, Time 16, Duration: 2 hours
Activity: Relax, Priority: 1, Scheduled Start: Day 1, Time 20, Duration: 3 hours
